# Proyek Analisis Data: Bike Sharing
- **Nama:** Syafiq Ziyadul Arifin
- **Email:** szarifin20041@gmail.com
- **ID Dicoding:** safiq53

## Menentukan Pertanyaan Bisnis

- Bagaimana kecenderungan users untuk rent bike sharing pada suatu weather tertentu?
- Bagaimana pengaruh season terhadap jumlah rent bike sharing, baik yang casual users, registered users, maupun total keduanya?

## Import Semua Packages/Library yang Digunakan

Import semua packages/library yang dibutuhkan/digunakan

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

: 

## Data Wrangling

### Gathering Data

Import data

In [ ]:
day_df = pd.read_csv('data/day.csv')
hour_df = pd.read_csv('data/hour.csv')

: 

Tampilkan 5 row pertama

In [ ]:
day_df.head()

: 

In [ ]:
hour_df.head()

: 

### Assessing Data

Mencari apakah ada data yang kosong

In [ ]:
day_df.isnull().sum()

: 

In [ ]:
hour_df.isnull().sum()

: 

Mencari apakah ada data yang duplikat

In [ ]:
day_df.duplicated().sum()

: 

In [ ]:
hour_df.duplicated().sum()

: 

Mendapatkan data outlier untuk setiap kolom

In [ ]:
# Select numeric columns from the DataFrame
day_numeric_columns = day_df.select_dtypes(include=np.number).columns
day_numeric_df = day_df[day_numeric_columns]

# Dictionary to store outliers for each numeric column
day_outliers_dict = {}

# Iterate over each numeric column to identify outliers
for column in day_numeric_columns:
    # Calculate the first and third quartiles
    q25, q75 = np.percentile(day_df[column], 25), np.percentile(day_df[column], 75)

    # Calculate the interquartile range (IQR)
    iqr = q75 - q25

    # Define the cutoff values for outliers
    cut_off = iqr * 1.5
    day_minimum, day_maximum = q25 - cut_off, q75 + cut_off

    # Identify outliers for the current column
    day_outliers = day_df[(day_df[column] < day_minimum) | (day_df[column] > day_maximum)][column]

    # Store outliers in the dictionary
    day_outliers_dict[column] = day_outliers

# Print the identified outliers for each column
for column, outliers_series in day_outliers_dict.items():
    if not outliers_series.empty:
        print(outliers_series)
        print("\n")

: 

In [ ]:
hour_numeric_columns = hour_df.select_dtypes(include=np.number).columns
hour_numeric_df = hour_df[hour_numeric_columns]

hour_outliers_dict = {}

for column in hour_numeric_columns:
    q25, q75 = np.percentile(hour_df[column], 25), np.percentile(hour_df[column], 75)
    iqr = q75 - q25
    cut_off = iqr * 1.5
    hour_minimum, hour_maximum = q25 - cut_off, q75 + cut_off

    hour_outliers = hour_df[(hour_df[column] < hour_minimum) | (hour_df[column] > hour_maximum)][column]
    hour_outliers_dict[column] = hour_outliers

for column, outliers_series in hour_outliers_dict.items():
    if not outliers_series.empty:
        print(outliers_series)
        print("\n")

: 

### Cleaning Data

Menghapus data outlier untuk setiap kolom

In [ ]:
# Create a copy of the original DataFrame for filtering outliers
day_df_filtered = day_df.copy()

# Iterate over each column and its corresponding outliers
for column, outliers_series in day_outliers_dict.items():
    # Check if the column has outliers
    if not outliers_series.empty:
        # Get the indices of the outliers
        outliers_index = outliers_series.index

        # Identify duplicated indices among outliers
        duplicates_mask = outliers_index.duplicated(keep='first')

        # Retain only the unique indices (keep the first occurrence)
        unique_outliers_index = outliers_index[~duplicates_mask]

        # Drop rows with unique outlier indices from the filtered DataFrame
        day_df_filtered = day_df_filtered.drop(index=unique_outliers_index, errors='ignore')

# The resulting day_df_filtered will have outliers removed based on the specified logic

: 

In [ ]:
hour_df_filtered = hour_df.copy()

for column, outliers_series in hour_outliers_dict.items():
    if not outliers_series.empty:
        outliers_index = outliers_series.index
        duplicates_mask = outliers_index.duplicated(keep='first')
        unique_outliers_index = outliers_index[~duplicates_mask]
        hour_df_filtered = hour_df_filtered.drop(index=unique_outliers_index, errors='ignore')

: 

\## Exploratory Data Analysis (EDA)

Mendapatkan statistik umum dari setiap kolom

In [ ]:
day_df.describe()

: 

In [ ]:
hour_df.describe()

: 

## Visualization & Explanatory Analysis

### Pertanyaan 1:

In [ ]:
# Define your custom palette
custom_palette = sns.color_palette("pastel", n_colors=3)

# Set the style and context for the plot
sns.set(style="whitegrid")
plt.figure(figsize=(10, 6))

# Visualize the number of bike rentals for each weather condition
ax = sns.barplot(x='weathersit', y='cnt', data=day_df_filtered, estimator=sum, errorbar=None, palette=custom_palette, dodge=False, hue='weathersit', legend=False)

# Add title and labels
plt.title('Jumlah Penyewaan Sepeda berdasarkan Kondisi Cuaca', fontsize=16)
plt.xlabel('Kondisi Cuaca', fontsize=14)
plt.ylabel('Jumlah Penyewaan Sepeda', fontsize=14)

# Add value annotations on top of the bars
for p in ax.patches:
    ax.annotate(f'{p.get_height():.0f}', (p.get_x() + p.get_width() / 2., p.get_height()), ha='center', va='baseline', fontsize=12, color='black')

# Show the plot
plt.show()

day_df_filtered[['weathersit', 'cnt']].to_csv('question_1.csv', index=False)

: 

Berdasarkan grafik di atas, terlihat bahwa jumlah penyewaan sepeda mencapai puncak tertingginya ketika kondisi cuaca adalah Clear, Few clouds, atau Partly cloudy. Sebaliknya, jumlah penyewaan sepeda mencapai titik terendah, dengan nilai 0, saat kondisi cuaca adalah Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog. Kesimpulan ini dapat diambil dari perbandingan tinggi rendahnya batang-batang pada grafik untuk setiap kondisi cuaca yang ditampilkan.

### Pertanyaan 2:

In [ ]:
# Set style
sns.set(style="whitegrid")

# Average for each season
seasonal_analysis = day_df_filtered.groupby('season')[['casual', 'registered', 'cnt']].mean()

# Visualization
plt.figure(figsize=(14, 8))

# Subplot 1
plt.subplot(2, 2, 1)
sns.barplot(x=seasonal_analysis.index, y=seasonal_analysis['casual'], color='skyblue')
plt.title('Jumlah Casual Users per Musim')
plt.xlabel('Musim')
plt.ylabel('Jumlah Casual Users')
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Subplot 2
plt.subplot(2, 2, 2)
sns.barplot(x=seasonal_analysis.index, y=seasonal_analysis['registered'], color='lightgreen')
plt.title('Jumlah Registered Users per Musim')
plt.xlabel('Musim')
plt.ylabel('Jumlah Registered Users')
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Subplot 3
plt.subplot(2, 2, 3)
sns.barplot(x=seasonal_analysis.index, y=seasonal_analysis['cnt'], color='salmon')
plt.title('Jumlah Total Users per Musim')
plt.xlabel('Musim')
plt.ylabel('Jumlah Total Users')
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Adjust layout for better spacing
plt.tight_layout()

# Show the plot
plt.show()

: 

Dari grafik tersebut terlihat bahwa jumlah perental sepeda mencapai puncak tertinggi pada musim Fall, diikuti oleh musim Summer, Winter, dan Spring.

## Conclusion

- Conclution pertanyaan 1

  Berdasarkan perbandingan tinggi rendahnya bar-bar pada grafik, dapat disimpulkan bahwa jumlah penyewaan sepeda mencapai puncak tertingginya ketika kondisi cuaca adalah Clear, Few clouds, atau Partly cloudy. Sebaliknya, jumlah penyewaan sepeda mencapai titik terendah, dengan nilai 0, saat kondisi cuaca adalah Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog.

  Hal ini menunjukkan bahwa kondisi cuaca yang bersih, cerah, dan sebagian berawan lebih mendukung aktivitas bersepeda, sedangkan kondisi cuaca yang buruk seperti hujan deras, petir, kabut, salju, dan kabut dapat menghambat atau bahkan mencegah orang untuk menyewa sepeda.

  Dengan demikian, pemilik atau operator penyewaan sepeda dapat menggunakan informasi ini untuk mengoptimalkan layanan mereka, seperti meningkatkan promosi atau ketersediaan sepeda selama cuaca cerah dan mengurangi ekspektasi selama kondisi cuaca buruk.

- Conclution pertanyaan 2

  Berdasarkan grafik, dapat disimpulkan bahwa jumlah perental sepeda mencapai puncak tertinggi pada musim Fall, diikuti oleh musim Summer, Winter, dan Spring. Hal ini mungkin disebabkan oleh suhu rata-rata yang cukup nyaman pada musim gugur. Suhu yang tidak terlalu panas seperti musim panas atau terlalu dingin seperti musim dingin membuat kondisi cuaca menjadi lebih menyenangkan untuk bersepeda.

  Pada musim gugur, banyak orang mungkin lebih tertarik untuk melakukan aktivitas di luar ruangan, termasuk bersepeda, karena suhu yang lebih sejuk dan cuaca yang stabil. Sebaliknya, musim panas, meskipun memiliki jumlah perental sepeda yang tinggi, mungkin memiliki suhu yang lebih tinggi dan potensi kelembapan yang dapat mempengaruhi tingkat kenyamanan bersepeda.

  Kondisi cuaca yang menyenangkan pada musim gugur menjadi faktor utama yang mendorong banyak orang untuk menyewa sepeda, dan pemahaman ini dapat membantu penyedia layanan sepeda untuk mengoptimalkan strategi pemasaran dan penyediaan sepeda selama musim-musim tertentu.